<a href="https://colab.research.google.com/github/kozeljko/nlp-models/blob/master/gab_and_reddit_glove300_sigmoid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Init environment

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
# How I got Glove6B embeddings
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!ls
#!unzip glove.6B.zip
#!mkdir drive/MyDrive/nlp/models-pre/glove6B
#!mv *d.txt drive/MyDrive/nlp/models-pre/glove6B

In [5]:
!pip install lemmagen3 emoji
!pip install --upgrade keras

import sys
sys.path.append('drive/MyDrive/nlp/nlp-offensive-language/src')


from preprocessing import preprocess



     |████████████████████████████████| 12.4MB 223kB/s 
     |████████████████████████████████| 133kB 32.8MB/s 
     |████████████████████████████████| 194kB 42.8MB/s 
Requirement already up-to-date: keras in /usr/local/lib/python3.7/dist-packages (2.4.3)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
DATASETS_DIR = "drive/MyDrive/nlp/nlp-offensive-language/datasets/"

GLOVE_DIR = "drive/MyDrive/nlp/models-pre/glove6B/"

In [38]:
import os, csv
import numpy as np
import pandas as pd
from preprocessing import *

#filename = "english/embeddia/dataset.csv"
filename = "english/gab_and_reddit/dataset.csv"
csv_read = csv.reader(open(os.path.join(DATASETS_DIR, filename), encoding="utf8"), delimiter=",")

texts = []
labels = []
for line in csv_read:
  if line[0] == "id":
    continue

  text = line[1]
  text = preprocess(text, [PP_REMOVE_USERNAME_HANDLES, PP_REMOVE_URLS, PP_REMOVE_SPECIAL_CHARACTERS, PP_REMOVE_BASE_PUNCTUATIONS])
  text = " ".join(text)

  texts.append(text)
  if line[2] == "OFF":
    labels.append([1])
  else:
    labels.append([0])

print("Loaded dataset")
print(str(len(texts)) + " texts")
print("First: " + texts[0])
print(labels[0])

Loaded dataset
56085 texts
First: i joined gab to remind myself how retarded jew haters are You would not be typing on your abacus without them you retard
[1]


In [39]:
import os
import numpy as np
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Prepare tokenizer given loaded texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
seq = tokenizer.texts_to_sequences(texts)

vocab_size = len(tokenizer.word_index)+1

# Load Glove model
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = 100
pad_seq = pad_sequences(seq,maxlen=MAX_SEQUENCE_LENGTH)
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# Create embedding matrix

word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# Create embedding layer

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

# Created embedding layer

Found 400000 word vectors.


Train model

In [75]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Embedding,Bidirectional
from keras.metrics import Precision, Recall


train_index = int(len(pad_seq) * 0.8)

train_seq = np.array(pad_seq[:train_index])
train_labels = np.array(labels[:train_index])

test_seq = np.array(pad_seq[train_index:])
test_labels = np.array(labels[train_index:])

model = Sequential()
model.add(embedding_layer)
model.add(Dense(150,activation = 'relu'))
model.add(LSTM(128))
model.add(Dense(50,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['accuracy'])

model.fit(train_seq, train_labels, epochs=10, validation_split=0.3, batch_size=128)

Epoch 1/10
246/246 [==============================] - 7s 22ms/step - loss: 0.4610 - accuracy: 0.7898 - val_loss: 0.3489 - val_accuracy: 0.8783
Epoch 2/10
246/246 [==============================] - 5s 19ms/step - loss: 0.3071 - accuracy: 0.8916 - val_loss: 0.3547 - val_accuracy: 0.8729
Epoch 3/10
246/246 [==============================] - 5s 19ms/step - loss: 0.2874 - accuracy: 0.8984 - val_loss: 0.3405 - val_accuracy: 0.8822
Epoch 4/10
246/246 [==============================] - 5s 19ms/step - loss: 0.2716 - accuracy: 0.9025 - val_loss: 0.3363 - val_accuracy: 0.8772
Epoch 5/10
246/246 [==============================] - 5s 19ms/step - loss: 0.2612 - accuracy: 0.9050 - val_loss: 0.3464 - val_accuracy: 0.8782
Epoch 6/10
246/246 [==============================] - 5s 19ms/step - loss: 0.2496 - accuracy: 0.9077 - val_loss: 0.3342 - val_accuracy: 0.8903
Epoch 7/10
246/246 [==============================] - 5s 20ms/step - loss: 0.2399 - accuracy: 0.9092 - val_loss: 0.3465 - val_accuracy: 0.8853

Evaluate model

In [76]:
# Evaluate
hm = model.evaluate(test_seq, test_labels, verbose=0, return_dict=True)
print(hm)
print('Test accuracy:', hm['accuracy'])

{'loss': 0.42031413316726685, 'accuracy': 0.866631031036377}
Test accuracy: 0.866631031036377
